In [ ]:
from constants import ROCAR_CSV
from langdetect import detect
import pandas as pd
from tqdm import tqdm

df = pd.read_csv(ROCAR_CSV)

In [ ]:
from collections import defaultdict

languages = {}

texts = df["input"].astype(str)
failed_indexes = []
language_indexes = defaultdict(list)

for i, text in enumerate(tqdm(texts)):
    try:
        if text == "nan":
            continue
        detected = detect(text)
        language_indexes[detected].append(i)
        if detected in languages.keys():
            languages[detected] += 1
        else:
            languages[detected] = 1
    except Exception as e:
        print(e)
        failed_indexes.append(i)

print(languages)
print(failed_indexes)
print(language_indexes)

# {'ro': 29605, 'tl': 1075, 'en': 619, 'it': 524, 'fr': 14, 'de': 5, 'ca': 27, 'hr': 1, 'pt': 3, 'af': 5, 'lt': 1, 'sl': 1, 'es': 3, 'sv': 1, 'tr': 1, 'id': 1, 'sq': 1, 'sk': 1, 'cy': 1, 'nl': 1}

In [ ]:
for k, v in language_indexes.items():
    if k != "ro":
        print(k)
        for value in v[:1]:
            print(df["description"][value])

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from core.src.constants import ROCAR_CSV

sns.set(rc={"figure.figsize": (10, 10)})
plt.figure(figsize=(10, 8))
df = pd.read_csv(ROCAR_CSV)

In [ ]:
print(df["input"][0])

print(df["input"].apply(len).mean())
df["input"].apply(len).hist()
plt.show()

In [ ]:
# replace phone numbers with [TEL] and email addresses with [EMAIL], also replace html tags with [HTML]
import re


def replace_patterns(text: str):
    email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
    phone_pattern = r"\b\d{3}[-.]?\d{3}[-.]?\d{4}\b"
    html_pattern = r"<.*?>"
    text = re.sub(email_pattern, "[EMAIL]", text)
    text = re.sub(phone_pattern, "[TEL]", text)
    text = re.sub(html_pattern, "[HTML]", text)
    return text


df["input"] = df["input"].apply(replace_patterns)

In [ ]:
import emoji


def replace_emojis(text: str):
    return emoji.demojize(text, delimiters=("[", "]"))


df["input"] = df["input"].apply(replace_emojis)

In [ ]:
from wordcloud import WordCloud

all_descriptions = " ".join(df["input"])

wordcloud = WordCloud(width=800, height=400, background_color="white").generate(all_descriptions)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
df.to_csv("nlp.csv", index=False)